In [3]:
!pip install -q langchain langchain-core langchain-huggingface python-dotenv

In [1]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [5]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
load_dotenv()
import os


In [6]:
# Initialize the Hugging Face model endpoint
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation",
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN")  # ✅ explicitly passed
)

# Wrap it in a Chat interface
model = ChatHuggingFace(llm=llm)

In [7]:
# schema
class Person(BaseModel):
  name: str = Field(description="person's name")
  age: int = Field(gt = 18, description="person's age")
  city: str = Field(description="person's city")

In [8]:
# parser
parser = PydanticOutputParser(pydantic_object=Person)

In [9]:
# template
template = PromptTemplate(
    template='Generate name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

In [18]:
# prompt
prompt = template.format_prompt(place='India')
print(prompt)

text='Generate name, age and city of a fictional India person \n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "person\'s name", "title": "Name", "type": "string"}, "age": {"description": "person\'s age", "exclusiveMinimum": 18, "title": "Age", "type": "integer"}, "city": {"description": "person\'s city", "title": "City", "type": "string"}}, "required": ["name", "age", "city"]}\n```'


In [19]:
result = model.invoke(prompt)
print(result)

content='```json\n{"name": "Arundhati Gupta", "age": 25, "city": "Bangalore"}\n```' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 240, 'total_tokens': 268}, 'model_name': 'google/gemma-2-2b-it', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run--d59b68d2-eb51-4c7d-9c4f-5d323552f89b-0' usage_metadata={'input_tokens': 240, 'output_tokens': 28, 'total_tokens': 268}


In [20]:
final_result = parser.parse(result.content)
print(final_result)

name='Arundhati Gupta' age=25 city='Bangalore'
